In [237]:
import pinecone
import tqdm

pinecone.init(      
	api_key='2906808a-8423-49a6-a6ed-554d7094d82c',      
	environment='gcp-starter'      
)      
index = pinecone.Index('jd-builder')

import os
from dotenv import load_dotenv

load_dotenv("C:\\Users\\abuza\\Desktop\\jobd\\api.env")
API_KEY = os.getenv("API_KEY")
os.environ['OPENAI_API_KEY'] = API_KEY

In [238]:
import openai
import os
response = openai.Embedding.create(
  input="Educative answers section is helpful",
  model="text-embedding-ada-002"
)

print(len(response['data'][0]['embedding']))

1536


In [178]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
def read_and_separate_roles(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
    
    # Split the data into individual roles based on two newline characters
    role = data.split('\n\n')
    return role

def make_embeddings(string):
    embeddings = OpenAIEmbeddings()
    result = embeddings.embed_documents(texts=string)
    return result[0]

# Provide the file path to your text file
file_path = 'database.txt'

# Read and separate the roles
roles = read_and_separate_roles(file_path)

role = roles[1].split(':')[0]
print(role)
embed = make_embeddings(role)
len(embed)


MLOps Engineer


1536

In [179]:
roles

['Machine Learning Engineer:\n    Programming Skills such as Python/R/Java\n    Machine Learning Algorithms such as Supervised Learning, Unsupervised Learning, Reinforcement Learning\n    Deep Learning Frameworks such as TensorFlow, PyTorch, Keras\n    Cloud Platforms such as AWS, Microsoft Azure, Google Cloud\n    Database Knowledge\n    Statistics and Probability\n    Libraries such as Scikit-learn, NumPy, Pandas\n    Data Visualization Tools such as Matplotlib, Seaborn\n    Algorithm Building and Optimization\n    Model Evaluation and Optimization Techniques\n    Confusion Matrix\n    ROC-AUC\n    F1 Score\n    Hyperparameter Tuning Methods\n    Precision-Recall Curve\n    Grid Search\n    Random Search',
 'MLOps Engineer:\n    Programming Skills such as Python/R/Java\n    Machine Learning Familiarity\n    Machine Learnign Algorithms\n    DevOps Tools such as Docker, Kubernetes, Jenkins, Ansible\n    Cloud Platforms such as AWS, Microsoft Azure, Google Cloud\n    CI/CD Practices\n  

In [180]:
for i in range(len(roles)):
    role = roles[i].split(':')[0]
    index.upsert([
        (f'id-{i}', make_embeddings(roles[i]), {'role_title': role, 'description': roles[i]})
    ])

In [240]:
for i in range(len(roles)):
    index.delete(ids=[f'id-{i}'])


In [241]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [230]:
query = """ABOUT THE ROLE:

We are seeking a full-time Data Engineer III to join our team and define and deploy mission-critical data infrastructure. The primary responsibility for this role is defining the processing of large healthcare datasets which enable Rightway to generate insights that improve the health of more than 1 million members. Necessary skills for this role are expertise with the AWS ecosystem and proficiency in writing production-ready Python code. Exposure to medical data is a plus but not required.




WHAT YOU'LL DO:

Write production-level Python and SQL code
Use AWS services such as Lambda, Glue, and Fargate
Use Apache Airflow to build batch data pipelines
Use DBT to maintain Extract, Load, and Transform (ELT) processes
Design database schemas in PostgreSQL to optimize data integrity and performance




WHO YOU ARE:

4+ years of industry experience
Experience using AWS systems in a production environment such as Lambda and Glue
Have written SQL and Python code in a production environment
Has experience scaling data infrastructure to handle high volumes of data




EXTRA CREDIT:

Experience with medical and pharmacy claims data
Experience deploying applications using serverless architecture
Experience with Master Data Management and data governance
Use infrastructure-as-code services such as Terraform or Cloudformation




BASE SALARY: $125,000-$175,000

"""
json_format = index.query(
    make_embeddings(string=query),
    top_k=2,
    include_metadata=True
)

json_format

{'matches': [{'id': 'id-5',
              'metadata': {'description': 'Big Data Engineer:\n'
                                          '    Programming Skills such as '
                                          'Python/Java\n'
                                          '    Big Data Programming '
                                          'Technologies such asHadoop, '
                                          'MapReduce, Spark\n'
                                          '    Cloud Platforms such as AWS, '
                                          'Google Cloud, Microsoft Azure\n'
                                          '    Data Warehousing and ETL\n'
                                          '    SQL and NoSQL Databases\n'
                                          '    Real-Time Processing Frameworks '
                                          'such as Kafka, Flink\n'
                                          '    Linux/UNIX Proficiency\n'
                                          '

In [232]:
context = json_format['matches'][0]['metadata']['description']
print (context)
seniority_level = 'Entry'


Big Data Engineer:
    Programming Skills such as Python/Java
    Big Data Programming Technologies such asHadoop, MapReduce, Spark
    Cloud Platforms such as AWS, Google Cloud, Microsoft Azure
    Data Warehousing and ETL
    SQL and NoSQL Databases
    Real-Time Processing Frameworks such as Kafka, Flink
    Linux/UNIX Proficiency
    Big Data File Formats such as Avro, Parquet
    Docker and Kubernetes
    Distributed Systems Concepts


In [234]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-4', temperature=0.0)
llm.predict(f'''
Task: You are Mike and your role is to ask technical questions regarding the skills from the TEXT below
TEXT: 
{context}
Kindly check if the any of the skill is already discussed in the job desciption. Only unanswered questions needs to be asked
job_description:
{query}


Question Format: <<(Do the candidate needs to have experience with xyz...)>>
Output Format: JSON
key: "unanswered_questions" value: [<<(for example: Do the candidate needs to have experience with xyz)>>, ....]

Instructions:
- Ensure to ask questions based on the seniority level
- Ask the broad questions from the candidates in such a way that similar information is asked in one single question without dropping any single piece of information
- Make sure to ask the questions in the interrogative tone
- Make sure to ask the questions that are most relevant to the provided job descrition and role seniority level
- if the information is given in the job description about something do not ask it again
''')



'{\n"unanswered_questions": [\n"Do the candidates need to have experience with Big Data Programming Technologies such as Hadoop, MapReduce, and Spark?",\n"Do the candidates need to be proficient with Cloud Platforms other than AWS, such as Google Cloud and Microsoft Azure?",\n"Do the candidates need to have experience with Data Warehousing and ETL?",\n"Do the candidates need to have experience with NoSQL Databases?",\n"Do the candidates need to have experience with Real-Time Processing Frameworks such as Kafka and Flink?",\n"Do the candidates need to have Linux/UNIX Proficiency?",\n"Do the candidates need to have knowledge of Big Data File Formats such as Avro and Parquet?",\n"Do the candidates need to have experience with Docker and Kubernetes?",\n"Do the candidates need to have understanding of Distributed Systems Concepts?"\n]\n}'

In [223]:

answer = '{\n"unanswered_questions": [\n"Do the candidates need to have experience with Big Data Programming Technologies such as Hadoop, MapReduce, and Spark?",\n"Do the candidates need to be proficient with Cloud Platforms other than AWS, like Google Cloud and Microsoft Azure?",\n"Do the candidates need to have experience with Data Warehousing and ETL?",\n"Do the candidates need to have experience with NoSQL Databases, in addition to SQL?",\n"Do the candidates need to have experience with Real-Time Processing Frameworks such as Kafka and Flink?",\n"Do the candidates need to have Linux/UNIX Proficiency?",\n"Do the candidates need to have knowledge of Big Data File Formats such as Avro and Parquet?",\n"Do the candidates need to have experience with Docker and Kubernetes?",\n"Do the candidates need to have understanding of Distributed Systems Concepts?"\n]\n}'


In [224]:
import json
final = json.loads(answer)
print (final['unanswered_questions'])
len(final['unanswered_questions'])

['Do the candidates need to have experience with Big Data Programming Technologies such as Hadoop, MapReduce, and Spark?', 'Do the candidates need to be proficient with Cloud Platforms other than AWS, like Google Cloud and Microsoft Azure?', 'Do the candidates need to have experience with Data Warehousing and ETL?', 'Do the candidates need to have experience with NoSQL Databases, in addition to SQL?', 'Do the candidates need to have experience with Real-Time Processing Frameworks such as Kafka and Flink?', 'Do the candidates need to have Linux/UNIX Proficiency?', 'Do the candidates need to have knowledge of Big Data File Formats such as Avro and Parquet?', 'Do the candidates need to have experience with Docker and Kubernetes?', 'Do the candidates need to have understanding of Distributed Systems Concepts?']


9

In [236]:
llm.predict(f'''Using the questions written below and the seniority level
    <<{final['unanswered_questions']}>>
    role_seniority_level:
    <<{seniority_level}>>
Reduce the number of questions to 5 based on the seniority level. Think Critically for that!. Make sure to return a dictionery with a key precise_questions''')

'{\n"precise_questions": [\'Do the candidates need to have experience with Big Data Programming Technologies such as Hadoop, MapReduce, and Spark?\', \'Do the candidates need to be proficient with Cloud Platforms other than AWS, like Google Cloud and Microsoft Azure?\', \'Do the candidates need to have experience with Data Warehousing and ETL?\', \'Do the candidates need to have Linux/UNIX Proficiency?\', \'Do the candidates need to have understanding of Distributed Systems Concepts?\']\n}'